# Ensemble methods to compute participants' contribution 

In [1]:
import sys
sys.path.append("../")

In [2]:
from decentralized_smart_grid_ml.federated_learning.federated_aggregator import weighted_average_aggregation

2021-12-21 16:37:00.027516: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-21 16:37:00.027586: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
import pandas as pd
from tensorflow.keras.experimental import LinearModel

In [4]:
import numpy as np

In [5]:
train_set_path1 = "../data_sample/simple_ml_task/participants/participant_0/simple_ml_task_0.csv"
train_set_df1 = pd.read_csv(train_set_path1)
x_train1, y_train1 = train_set_df1[["x1", "x2"]].values[:10], train_set_df1["y"].values[:10]

In [6]:
train_set_path2 = "../data_sample/simple_ml_task/participants/participant_1/simple_ml_task_1.csv"
train_set_df2 = pd.read_csv(train_set_path2)
x_train2, y_train2 = train_set_df2[["x1", "x2"]].values, train_set_df2["y"].values

In [7]:
model1 = LinearModel(activation="sigmoid")
model1.compile(optimizer="sgd", loss="mse", metrics="accuracy")

model2 = LinearModel(activation="sigmoid")
model2.compile(optimizer="sgd", loss="mse", metrics="accuracy")

2021-12-21 16:38:18.153554: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-12-21 16:38:18.153582: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-12-21 16:38:18.153608: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Lenovo-ideapad): /proc/driver/nvidia/version does not exist
2021-12-21 16:38:18.153860: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
model1.fit(x_train1, y_train1, epochs=3)
model2.fit(x_train2, y_train2, epochs=3)

Epoch 1/3
1/1 [==============================] - 0s 248ms/step - loss: 0.2500 - accuracy: 0.3000
Epoch 2/3
1/1 [==============================] - 0s 2ms/step - loss: 0.2293 - accuracy: 0.7000
Epoch 3/3
1/1 [==============================] - 0s 2ms/step - loss: 0.2161 - accuracy: 0.7000
Epoch 1/3
107/107 [==============================] - 0s 796us/step - loss: 0.0881 - accuracy: 0.9874
Epoch 2/3
107/107 [==============================] - 0s 840us/step - loss: 0.0255 - accuracy: 0.9997
Epoch 3/3
107/107 [==============================] - 0s 888us/step - loss: 0.0157 - accuracy: 0.9997


In [9]:
models = [model1, model2]

In [11]:
validation_set_path = "../data_sample/simple_ml_task/validator/simple_ml_task_validation.csv"
val_set_df = pd.read_csv(validation_set_path)
x_val, y_val = val_set_df[["x1", "x2"]].values, val_set_df["y"].values

In [12]:
test_set_path = "../data_sample/simple_ml_task/validator/simple_ml_task_test.csv"
test_set_df = pd.read_csv(test_set_path)
x_test, y_test = test_set_df[["x1", "x2"]].values, test_set_df["y"].values

In [13]:
global_model = LinearModel(activation="sigmoid")
global_model.compile(optimizer="sgd", loss="mse", metrics="accuracy")
# here the fit function is called because it needs the build. The trained model will NOT be used becuase
# we will override its weights with the new ones
global_model.fit(x_train1, y_train1)

1/1 [==============================] - 0s 213ms/step - loss: 0.2500 - accuracy: 0.3000


In [14]:
participants_weights = []
for model in models:
    participants_weights.append(model.get_weights())

# Simple average of local models

In [ ]:
average_weights = weighted_average_aggregation(participants_weights, [0.5, 0.5])

In [ ]:
global_model.set_weights(average_weights)
simple_average_evaluation = global_model.evaluate(x_test, y_test)

# Ensamble method based on local models' output

In this approach we apply the softmax function to the output computed by each local models so that we can weight the contribution of each local model. 

In [ ]:
from scipy.special import softmax as sc_softmax

In [ ]:
score1 = model1.evaluate(x_test, y_test)[1]
score2 = model2.evaluate(x_test, y_test)[1]

In [ ]:
scores = [score1, score2]

In [ ]:
alpha = sc_softmax(scores)
print(alpha)

In [ ]:
ensamble_general_weights = weighted_average_aggregation(participants_weights, alpha)
global_model.set_weights(ensamble_general_weights)

In [ ]:
ensamble_general_evaluation = global_model.evaluate(x_test, y_test)

In [20]:
# load models from file
def load_all_models(participants_weights):
    all_models = []
    for participant_weights in participants_weights:
        model = LinearModel(activation="sigmoid")
        model.compile(optimizer="sgd", loss="mse", metrics="accuracy")
        model.fit(x_train1, y_train1)
        model.set_weights(participant_weights)
        all_models.append(model)
    return all_models

In [32]:
from numpy import dstack

In [33]:
def stacked_dataset(members, inputX):
    stackX = None
    for model in members:
        # make prediction
        yhat = model.predict(inputX, verbose=0)
        # stack predictions into [rows, members, probabilities]
        if stackX is None:
            stackX = yhat
        else:
            stackX = dstack((stackX, yhat))
    # flatten predictions to [rows, members x probabilities]
    stackX = stackX.reshape((stackX.shape[0], stackX.shape[1]*stackX.shape[2]))
    return stackX

In [42]:
def fit_stacked_model(members, inputX, inputy):
    # create dataset using ensemble
    stackedX = stacked_dataset(members, inputX)
    # fit standalone model
    model = LogisticRegression()
    model.fit(stackedX, inputy)
    return model

In [38]:
# make a prediction with the stacked model
def stacked_prediction(members, model, inputX):
    # create dataset using ensemble
    stackedX = stacked_dataset(members, inputX)
    # make a prediction
    yhat = model.predict(stackedX)
    return yhat

In [39]:
models = load_all_models(participants_weights)

1/1 [==============================] - 0s 208ms/step - loss: 0.2500 - accuracy: 0.3000


In [40]:
for model in models:
    _, acc = model.evaluate(x_test, y_test, verbose=0)
    print('Model Accuracy: %.3f' % acc)

Model Accuracy: 0.490
Model Accuracy: 1.000


In [41]:
from sklearn.metrics import accuracy_score

In [45]:
from sklearn.linear_model import LogisticRegression

In [46]:
model = fit_stacked_model(models, x_test, y_test)
yhat = stacked_prediction(models, model, x_test)
acc = accuracy_score(y_test, yhat)
print('Stacked Test Accuracy: %.3f' % acc)

Stacked Test Accuracy: 1.000


# IMPORTANT:

Read here https://machinelearningmastery.com/weighted-average-ensemble-for-deep-learning-neural-networks/

# Comparison of aggregation methods

In [ ]:
print("Simple average evaluation: %s" % simple_average_evaluation)
print("Ensamble general evaluation: %s" % ensamble_general_evaluation)